In [1]:
#逆向最大匹配
class IMM(object):
    def __init__(self, dic_path):
        self.dictionary = set()
        self.maximum = 0
        #读取词典
        with open(dic_path, 'r', encoding='utf8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                if len(line) > self.maximum:
                    self.maximum = len(line)
    def cut(self, text):
        result = []
        index = len(text)
        while index > 0:
            word = None
            for size in range(self.maximum, 0, -1):
                if index - size < 0:
                    continue
                piece = text[(index - size):index]
                if piece in self.dictionary:
                    word = piece
                    result.append(word)
                    index -= size
                    break+
            if word is None:
                index -= 1
        return result[::-1]

def main():
    text = "南京市长江大桥"
    
    tokenizer = IMM('./data/imm_dic.utf8')
    print(tokenizer.cut(text))

main()

['南京市', '长江大桥']


In [3]:
#正向最大匹配
class IMM(object):
    def __init__(self, dic_path):
        self.dictionary = set()
        self.maximum = 0
        #读取词典
        with open(dic_path, 'r', encoding='utf8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                self.dictionary.add(line)
                if len(line) > self.maximum:
                    self.maximum = len(line)
    def cut(self, text):
        result = []
        index = 0
        while index < len(text):
            word = None
            for size in range(self.maximum, 0, -1):
                if len(text) - index < size:
                    continue
                piece = text[index: (index + size)]
                if piece in self.dictionary:
                    word = piece
                    result.append(word)
                    index += size
                    break
            if word is None:
                index += 1
        return result

def main():
    text = "南京市长江大桥"
    
    tokenizer = IMM('./data/imm_dic.utf8')
    print(tokenizer.cut(text))

main()

['南京市长', '大桥']


In [20]:
class HMM(object):
    def __init__(self):
        import os
        
        # 主要是用于存取算法中间结果，不用每次都训练模型
        self.model_file = './data/hmm_model.pkl'
        
        # 状态值集合
        self.state_list = ['B','M', 'E', 'S']
        
        # 参数加载，用于判断是否需要重新加载model_file
        self.load_para = False
    
    # 用于加载已计算的中间结果，当需要重新训练时，需初始化清空结果
    def try_load_model(self, trained):
        if trained:
            import pickle
            with open(self.model_file, 'rb') as f:
                self.A_dic = pickle.load(f)
                self.B_dic = pickle.load(f)
                self.Pi_dic = pickle.load(f)
                self.load_para = True
        else:
            # 状态转移概率（状态->状态的条件概率）
            self.A_dic = {}
            # 发射概率（状态->词语的条件概率）
            self.B_dic = {}
            # 状态的初始概率
            self.Pi_dic = {}
            self.load_para = False
    
    # 技术按转移概率、发射概率以及初始概率
    def train(self, path):
        
        # 重置几个概率矩阵
        self.try_load_model(False)
        
        # 统计状态出现次数，求p(o)
        Count_dic = {}
        
        # 初始化参数
        def init_parameters():
            for state in self.state_list:
                self.A_dic[state] = {s: 0.0 for s in self.state_list}
                self.Pi_dic[state] = 0.0
                self.B_dic[state] = {}
                Count_dic[state] = 0
        
        def makeLabel(text):
            out_text = []
            if len(text) == 1:
                out_text.append('S')
            else:
                out_text += ['B'] + ['M']*(len(text) -2) + ['E']
            
            return out_text
        
        init_parameters()
        line_num = -1
        # 观察者集合，主要是字以及标点等
        words = set()
        with open(path, encoding='utf8') as f:
            for line in f:
                # AJ注：在文件中一行的例子为'十亿 中华 儿女 踏上 新 的 征 程 。 '
                line_num += 1
                
                line = line.strip()
                if not line:
                    continue
                
                word_list = [i for i in line if i != ' '] # AJ注：一行包括哪些字
                words |= set(word_list) # 更新字的集合
                
                linelist = line.split()
                line_state = [] # AJ注：一行标注的结果
                for w in linelist:
                    line_state.extend(makeLabel(w))
                        
                assert len(word_list) == len(line_state)
                
                for k, v in enumerate(line_state):
                    Count_dic[v] += 1
                    if k == 0:
                        self.Pi_dic[v] += 1 # 每个句子的第一个字的状态，用于计算初始状态概率
                    else:
                        self.A_dic[line_state[k-1]][v] += 1 # 计算转移概率
                        self.B_dic[line_state[k]][word_list[k]] = \
                              self.B_dic[line_state[k]].get(word_list[k], 0) + 1.0 # 计算发射概率
        print('self.pi_dic.items', self.Pi_dic.items())
        self.Pi_dic = {k: v * 1.0 / line_num for k, v in self.Pi_dic.items()}
        self.A_dic = {k: {k1: v1 / Count_dic[k] for k1, v1 in v.items()}
                      for k, v in self.A_dic.items()}
        self.B_dic = {k: {k1: (v1 + 1) / Count_dic[k] for k1, v1 in v.items()}
                      for k, v in self.B_dic.items()}
        
        import pickle
        with open(self.model_file, 'wb') as f:
            pickle.dump(self.A_dic, f)
            pickle.dump(self.B_dic, f)
            pickle.dump(self.Pi_dic, f)

        return self
    
    def viterbi(self, text, states, start_p, trans_p, emit_p):
        V = [{}]
        path = {}
        for y in states:
            V[0][y] = start_p[y] * emit_p[y].get(text[0], 0)
            path[y] = [y]
        for t in range(1, len(text)):
            V.append({})
            newpath = {}
            
            #检验训练的发射概率矩阵中是否有该字
            neverSeen = text[t] not in emit_p['S'].keys() and \
                text[t] not in emit_p['M'].keys() and \
                text[t] not in emit_p['E'].keys() and \
                text[t] not in emit_p['B'].keys()
            for y in states:
                emitP = emit_p[y].get(text[t], 0) if not neverSeen else 1.0 #设置未知字单独成词
                (prob, state) = max(
                [(V[t-1][y0] * trans_p[y0].get(y, 0) * emitP, y0)
                for y0 in states if V[t - 1][y0] > 0])
                V[t][y] = prob
                newpath[y] = path[state] + [y]
            path = newpath
        
        if emit_p['M'].get(text[-1], 0) > emit_p['S'].get(text[-1], 0):
            (prob, state) = max([(V[len(text) - 1][y], y) for y in ('E','M')])
        else:
            (prob, state) = max([(V[len(text) - 1][y], y) for y in states])
                    
        return (prob, path[state])
    
    def cut(self, text):
        import os
        if not self.load_para:
            self.try_load_model(os.path.exists(self.model_file))
        prob, pos_list = self.viterbi(text, self.state_list, self.Pi_dic, self.A_dic, self.B_dic)      
        begin, next = 0, 0
        for i, char in enumerate(text):
            pos = pos_list[i]
            if pos == 'B':
                begin = i
            elif pos == 'E':
                yield text[begin: i+1]
                next = i+1
            elif pos == 'S':
                yield char
                next = i+1
        if next < len(text):
            yield text[next:]

In [21]:
hmm = HMM()
hmm.train('./data/trainCorpus.txt_utf8')

text = '这是一个非常棒的方案！'
res = hmm.cut(text)
print(text)
print(str(list(res)))

self.pi_dic.items dict_items([('B', 173416.0), ('M', 0.0), ('E', 0.0), ('S', 124543.0)])
这是一个非常棒的方案！
['这是', '一个', '非常', '棒', '的', '方案', '！']


### AJ: 测试一些东西

In [2]:
string = '十亿 中华 儿女 踏上 新 的 征 程 。 '
print(string.split())

['十亿', '中华', '儿女', '踏上', '新', '的', '征', '程', '。']


In [10]:
import pickle

with open('./data/hmm_model.pkl', 'rb') as f:
    dt1 = pickle.load(f)
    print(dt1)
    print('----------------------------')
    dt2 = pickle.load(f)
#     print(dt2[:50])
    print('----------------------------')
    dt3 = pickle.load(f)
    print(dt3)

{'S': {'S': 0.46460125869921165, 'M': 0.0, 'B': 0.3503213832274479, 'E': 0.0}, 'M': {'S': 0.0, 'M': 0.2777743117140081, 'B': 0.0, 'E': 0.7222256882859919}, 'B': {'S': 0.0, 'M': 0.1167175117318146, 'B': 0.0, 'E': 0.8832824882681853}, 'E': {'S': 0.5310673430644739, 'M': 0.0, 'B': 0.46893265693552616, 'E': 0.0}}
----------------------------
----------------------------
{'S': 0.41798844132394497, 'M': 0.0, 'B': 0.5820149148537713, 'E': 0.0}


# jieba

In [4]:
# 读取数据
def get_content(path):
    
    with open(path, 'r', encoding='gbk', errors='ignore') as f:
        content = ''
        for l in f:
            l = l.strip()
            content += l
    return content

# 定义高频词统计的函数
def get_TF(words, topK=10):
    tf_dic = {}
    for w in words:
        tf_dic[w] = tf_dic.get(w, 0) + 1
    return sorted(tf_dic.items(), key = lambda x: x[1], reverse=True)[:topK]

def main():
    import glob
    import random
    import jieba
    
    files = glob.glob('./data/news/C000013/*.txt')
    corpus = [get_content(x) for x in files]
    
    sample_inx = random.randint(0, len(corpus))
    split_words = list(jieba.cut(corpus[sample_inx]))
    print('样本之一：' + corpus[sample_inx])
    print('样本分词效果： '+ '/ '.join(split_words))
    print('样本top10词： '+ str(get_TF(split_words)))

main()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\47849\AppData\Local\Temp\jieba.cache
Loading model cost 1.031 seconds.
Prefix dict has been built succesfully.


样本之一：新华网香港4月27日电 李嘉诚基金会已累计捐款1．1亿元人民币在内地推广“宁养服务计划”，关怀晚期癌症患者，目前已服务病人4万余人，累计出诊达到42．7万次。据香港媒体报道，汕头大学副校长兼医学院院长李玉光介绍说，1998年3月与李嘉诚在香港谈及如何解决晚期癌症患者问题，初步酝酿了相关计划。同年11月在汕头大学医学院第一附属医院成立了第一个专业机构，由李嘉诚亲自定名为“宁养院”。李玉光说，“宁养院”确定了“贫困、癌症、家居、免费”的八字方针，表明“宁养院”的服务对象是贫困的癌症病人；服务方式是免费登门服务，有别于住院治疗；服务内容包括药物止痛、心理辅导，帮助病人减轻疼痛折磨，树立尊严。李嘉诚基金会罗敏洁医生说，有感于一所“宁养院”的服务范围太小，从2001年1月起，基金会开始在内地实施“宁养服务计划”，5年间已经将“宁养院”扩展到20家，覆盖了内地130个区县。每个“宁养院”的服务半径为300公里，每年由李嘉诚基金会拨付120万元人民币。到目前为止，该计划已服务病人4万多人，累计出诊达42．7万次。（完）
样本分词效果： 新华网/ 香港/ 4/ 月/ 27/ 日电/  / 李嘉诚基金会/ 已/ 累计/ 捐款/ 1/ ．/ 1/ 亿元/ 人民币/ 在/ 内地/ 推广/ “/ 宁养/ 服务/ 计划/ ”/ ，/ 关怀/ 晚期/ 癌症/ 患者/ ，/ 目前/ 已/ 服务/ 病人/ 4/ 万余/ 人/ ，/ 累计/ 出诊/ 达到/ 42/ ．/ 7/ 万次/ 。/ 据/ 香港媒体/ 报道/ ，/ 汕头大学/ 副校长/ 兼/ 医学院/ 院长/ 李玉光/ 介绍/ 说/ ，/ 1998/ 年/ 3/ 月/ 与/ 李嘉诚/ 在/ 香港/ 谈及/ 如何/ 解决/ 晚期/ 癌症/ 患者/ 问题/ ，/ 初步/ 酝酿/ 了/ 相关/ 计划/ 。/ 同年/ 11/ 月/ 在/ 汕头大学/ 医学院/ 第一/ 附属/ 医院/ 成立/ 了/ 第一个/ 专业/ 机构/ ，/ 由/ 李嘉诚/ 亲自/ 定名/ 为/ “/ 宁养院/ ”/ 。/ 李玉光/ 说/ ，/ “/ 宁养院/ ”/ 确定/ 了/ “/ 贫困/ 、/ 癌症/ 、/ 家居/ 、/ 免费/ ”/ 的/ 八字/ 方针/ ，/ 表明/ “/ 宁养院/ ”/ 的/ 服务/ 对象/ 是/ 贫困/ 的/ 癌症病人/ ；/ 服务/ 方式/

In [7]:
# 读取数据
def get_content(path):
    
    with open(path, 'r', encoding='gbk', errors='ignore') as f:
        content = ''
        for l in f:
            l = l.strip()
            content += l
    return content

# 定义高频词统计的函数
def get_TF(words, topK=10):
    tf_dic = {}
    for w in words:
        tf_dic[w] = tf_dic.get(w, 0) + 1
    return sorted(tf_dic.items(), key = lambda x: x[1], reverse=True)[:topK]

def stop_words(path):
    with open(path, encoding='utf8') as f:
        return [l.strip() for l in f]

def main():
    import glob
    import random
    import jieba
    
    files = glob.glob('./data/news/C000013/*.txt')
    corpus = [get_content(x) for x in files]
    
    sample_inx = random.randint(0, len(corpus))
    split_words = [x for x in jieba.cut(corpus[sample_inx]) if x not in stop_words('./data/stop_words.utf8')]
    print('样本之一：' + corpus[sample_inx])
    print('样本分词效果： '+ '/ '.join(split_words))
    print('样本top10词： '+ str(get_TF(split_words)))

main()

样本之一：慢性肾病易被忽视52岁的张女士，5年前因头晕去医院检查发现血压偏高，由于症状不明显，未引起足够重视。半年前，张女士在劳累后出现全身浮肿的现象，经检查发现血压高达180/100毫米汞柱，再到肾病专科就诊，B超检测显示肾脏已经萎缩，医生诊断为晚期肾病。虽经多方治疗但无明显效果，最终转为尿毒症。很多患者像张女士一样，初次到肾病专科就诊就被确诊为晚期肾病。据统计，有近1/3的患者在就诊时不知道已患有肾病，大多是因为出现心衰、贫血、消化道出血、感染等症状去医院检查时才查出尿毒症。这是因为肾脏有很强的解毒功能，当身体出现各种症状时，肾脏已出现严重损伤。此外，还有相当一部分早期肾病患者，由于讳疾忌医，不愿意进行彻底治疗，错过最佳治疗时间，导致了尿毒症。有资料显示，我国40岁以上的人肾病发病率约为8%%－9%%，每100万人口中有100人患有尿毒症，需要终生透析治疗，治疗费用是普通家庭难以承受的。体检可查出肾病除了从身体里清除废物和液体，肾脏还有其他重要功能：调节身体的水分和血液中的钠、钾、磷和钙等化学物质；清除摄入体内的药物和毒物；调节和生成血液中的激素；调节血压；生成红细胞；构建强壮的骨骼。肾脏功能受损时，废物会在血液里堆积到很高的水平而使您感觉不适，还会导致高血压(专题 访谈 咨询)、贫血、骨质脆弱、营养不良以及神经损伤等症状。肾病还会增加患心血管疾病的危险。大部分患者在疾病很严重之前不会有任何严重的症状。如果有下列症状就要引起注意：感觉乏力，精力不充沛；注意力难以集中；食欲下降；睡眠障碍；夜间抽筋；脚和踝部浮肿；眼周水肿，特别是早晨；皮肤干燥、痒；尿频，特别是夜间。除了要注意这些症状外，每年最好体检一次，尿检的一些指标可以反应肾脏的状况。学会呵护肾脏引起慢性肾病的原因主要有：肾炎，肾脏炎症会影响功能；糖尿病，血糖过高会损害身体内的许多器官，包括肾脏和心脏，还有血管、神经和眼睛；高血压，当血液对血管的压力增加时，如果不加控制或控制不良，就会引起心脏病、中风和肾脏疾病；胎儿在母亲体内发育畸形，如输尿管狭窄而阻碍尿液的正常流出，引起尿液返流到肾脏，引起感染而损伤肾脏；红斑狼疮及其他影响人体免疫系统的疾病；由肾结石、肿瘤(专题 访谈 咨询)或男性增大的前列腺而引起的尿路梗阻；反复的泌尿系统感染。预防肾病要对可能引起肾病的各种高风险因素如糖尿病、高血压病、高血脂等疾病进行及

In [16]:
#jieba词性标注示例
def main():
    import glob
    import random
    import jieba
    import jieba.posseg as psg
    
    files = glob.glob('./data/news/C000013/*.txt')
    corpus = [get_content(x) for x in files]
    
    sample_inx = random.randint(0, len(corpus))
    
    split_words = [w+'/'+t for w, t in psg.cut(corpus[sample_inx])
                  if w not in stop_words('./data/stop_words.utf8')]
    print('样本之一：'+corpus[sample_inx])
    print('样本分词效果：' + ' '.join(split_words))
    print('样本的topK（10）词：' + str(get_TF(split_words)))
main()

样本之一：&nbsp;静脉曲张是一种常见病，多发病于长期站立或负重工作的人，其主要原因是由静脉壁薄弱，静脉瓣膜损坏及静脉内压增高所造成。爱护美腿　　也因为一些身体原因或是客观工作环境等影响造成。比如：先天性静脉壁薄弱、或者有相关家族史、身体肥胖或者从事长久站立的工作、长期服用口服避孕药，女性怀孕身体压力过大等。目前来看，无论用什么方法都不能完全治愈静脉曲张。小编提示：随着子宫一天天增大，孕妈咪的下腔静脉受到挤压，影响了血液回流，从而使静脉扩张，血液淤积在皮下的血管中。它和下肢浮肿同样发生在孕晚期，以下肢和外阴的静脉曲张最为明显。&gt;&gt;&gt;全文静脉曲张在发生之初，往往没有明显的症状。有人会觉得小腿皮肤痒、腿部肿胀、酸痛、疲劳、腿部沈重感，特别是在经期刚要来时最严重，尤其是站了一整天下来，小腿特别酸痛、脚踝肿胀，把脚抬高就比较舒服。&nbsp;　　当静脉曲张越严重，疼痛就越明显，甚至发生脚部血液淤积，脚踝变紫色，更严重者，血液不易回流会发生色素沈淀、湿疹样皮肤炎，并可能产生郁积性溃疡。有时静脉破裂会出血。&nbsp;　　有时也会沿著静脉壁产生血块发炎，即血栓性静脉炎，表皮会沿著静脉呈现红肿、疼痛的症状，甚至可摸出非常疼痛的结节。若血栓跑到肺脏，就有可能发生致死的肺栓塞症。因此静脉曲张的治疗多以为了减轻症状，同时也要注意生活预防：·要避免久站或久坐不动。如果工作性质如此，要注意每小时活动一次，促进血液循环。·每天多做几次抬腿操。肌肉收缩，帮助腿部静脉将肢体远端的血液送回心脏，因此抬腿有助于使血液回流，减缓静曲张。·穿弹性裤袜。这样能帮助血液进入较大且较深处的静脉。·适当垫高床尾。睡觉的时候，把腿部抬高5-10厘米，有助于睡眠时的血液回流。·降低鞋后跟的高度。鞋跟过高，更多地增加脚趾的压力，阻碍回血。·保持理想体重。身体过于肥胖，增加了下肢的负担。·避免紧身衣物。以免使血液聚积在腿部。·小心服用避孕药。避孕药中的一些成分使血液的粘稠度过高，使血液流动缓慢，影响回流。·不抽烟；吸烟使血管收缩，回流缓慢。小编提示：长期保持腿部在一非常好的状态下，是预防腿部静脉曲张的重要事项。时刻注意保护您的腿，不使其有肿胀之情形，并注意以下12招&gt;&gt;&gt;&gt;全文点击进入下一页&gt;&gt;静脉曲张的常用治疗方法静脉曲张的常用治疗方法：治疗方法除要适当运动外，